# Import Tools

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

# Deeplearning tools
from tensorflow.keras import layers, models 
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D , MaxPool2D
from keras.layers import Activation, Dropout , BatchNormalization , Flatten , Dense
from keras.optimizers import Adam
from keras.utils import to_categorical

# Machine Learning Tools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

# Data Loading

In [25]:
import os
import cv2
from tqdm import tqdm

# Image paths
imagePaths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    #Only process files inside the 'images' folder
    if 'images' in dirname:
        for filename in filenames:
            if filename.endswith('.png'):
                imagePaths.append(os.path.join(dirname , filename))

In [16]:
import tqdm
import os
import cv2

Data = []
Target = []
resize = 150

cat = {'Viral Pneumonia':'Pneumonia' , 'Normal':'Normal' , 'COVID':'Covid-19'}

for imagePath in tqdm.tqdm(imagePaths):
    label = imagePath.split(os.path.sep)[-3]
    #Extract the label from the parent folder (2 levels up to )

    if label not in cat:
        continue
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)
    image = cv2.resize(image ,(resize , resize))/255.0  #Normalize image to [0 , 1]
    Data.append(image)
    Target.append(cat[label])

print(f"Processed  {len(Data)} images with corresponding labels.")

100%|██████████| 21165/21165 [00:52<00:00, 405.94it/s]  

Processed  15153 images with corresponding labels.


# Label Encoding and to one hot encoding
# example
[[0. ,1. ,0. ] , # 'Normal' -> [0, 1, 0]

[0. , 0. , 1. ], #'Pneumonia' -> [0, 0, 1]

[1. ,0. ,0. ]]# 'Covid-19' -> [1, 0, 0]

In [27]:
le = LabelEncoder()
labels = le.fit_transform(Target)
labels = to_categorical(labels)

print(le.classes_)
print(labels[0])

['Covid-19' 'Normal' 'Pneumonia']
[0. 1. 0.]


# Splitting Datase

In [29]:
X_train , x_test , y_train , y_test = train_test_split(Data, labels, test_size=0.2 , stratify=labels, random_state = 42)

# Further split the training data into training and validation sets
(x_train , x_val , y_train , y_val) = train_test_split(X_train, y_train, test_size=0.20, stratify=y_train , random_state=42)

trainX = np.array(x_train)
valX = np.array(x_val)
testX = np.array(x_test)
trainY = np.array(x_train)
valY = np.array(y_val)
testY = np.array(y_test)

print("Training data shape:", trainX.shape)
print("Validation data shape: ", valX.shape)
print('Testing data shape: ', testX.shape)
print("Traing labels shape: ", trainY.shape)

Training data shape: (9697, 150, 150, 3)
Validation data shape:  (2425, 150, 150, 3)
Testing data shape:  (3031, 150, 150, 3)
Traing labels shape:  (9697, 150, 150, 3)


# CNN Modal Training

In [1]:
from tensorflow.keras.layers import Input

2026-02-11 02:55:30.875142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770778531.135260     120 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770778531.200858     120 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770778531.759996     120 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770778531.760048     120 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770778531.760051     120 computation_placer.cc:177] computation placer alr

In [2]:
s = 150

model= Sequential()

# we have two parts in cnn
# feature learning

model.add(Input(shape=(s, s, 3))) 
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3) , activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(1,1)))
model.add(Dropout(0.25))


NameError: name 'Sequential' is not defined

In [36]:

# ann = dense layers

# Nueral networks (Dense Layers)
model.add(Flatten())
model.add(Dense(64 , activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation="softmax"))


In [ ]:

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.categorical_crossentropy,
              metrics= ['accuracy'])

# Train the model with validation data
epochs = 25
history = model.fit(trainX , trainY, epochs=epochs, batch_size=40, verbose=1, validation_data=(valX ,valY))

# Extract accuracy and loss deom the History Object

In [ ]:
history.history

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12,5))

plt.plot(1 , 2, 1)
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train' , 'Validation'])

# plot training & validation loss value
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history,history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train' , 'Validation'])

plt.tight_layout()
plt.show()

# Plotting Confusion Matrix

In [ ]:
y_pred = model.perdict(testX)
y_pred

In [ ]:
y_pred = np.argmax(predictions , axis=1)

# Convert testY from one-hot encoded to class labels (integers)
y_true = np.argmax(testY , axis=1)

# Convert confusion matrix
cm = confusion_matrix(y_true , y_pred)

print(cm)

In [ ]:
class_names = le.classes

In [ ]:
# create DataFrames for the confusion matrix
Confusion_Matrix = pd.DataFrame(cm , index=class_names , columns=class_names)

# Plot the confusion matrix using seaborn's matrix
plt.figure(figsize=(8, 7))
sns.heatmap(Confusion_Matrix , annot=True, fmt='d' , cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Condusion Matrix')
plt.show()




# Classification report on test


In [ ]:
# predIdxs = np.argmax(predIdxs , axis=1)
print(classification_report(y_true, y_pred , target_names=le.classes_, digits=5))

# Save Model and Label Encoder

In [ ]:
model.save('CNN_covid19_Xray_Version.h5') 

import pickle
pickle.dump(le , open("Label encoder.pk1" ,  'rb'))
print('saved')


# Load for inference

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('CNN_Convid19_Xray_Version.h5')

le = pickle.load(open("Label_encoder.pk1" , "rb"))

 # Real Time Detection System

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def detection_system(image_path , model , label_encoder , image_size=150):
    """
    Detection system function to classify an input image.

    Parameters:
    - image_path : the path to the image to classify
    - model : the trained Keras model.
    - label_encoder: the LabelEncoder used to encode the labels.
    - image_size: the target size for resizing the image (default: 150*150)

    Returns: 
    - predixted_label: the predicted class label.
    - confidence_Score = confidence score - predicted class
    """
    
    # load images
    image = cv2.imread(image_path)
    
    if image in None:
        raise ValueError(f"Image not found on this path, {image_path}")
    
    # Preprocessing
    image_rgb = cv2.cvColor(image , cv2.COLOR_BGR2RGB)
    
    image_resized = cv2.resize(image_rgb , {image_size , image_size})
    
    image_normalized = image_resized/ 255.0
    
    image_input = np.expand_dims(image_normalized ,axis=0)
    
    predictions = model.predict(image_input)
    
    predicted_index = np.argmax(predictions)
    confidence_score = predictions[0][predicted_index]
    
    label = label_encoder.inverse_tranform([predicted_index])[0]

return predixted_label , confidence_score


In [ ]:
# Example usage
image_path = '/kaggle/input/covid19-radiography-database/COVID19_Radiography_Dataset/Normal/images/'
predicted_label , confidence_score = detection_system(image_path , model, le)
print(f"Predicted Label: {Predicted_label} , Confidence_score: {Confidence_score * 100 : 2f}")